In [1]:
# numero_obligaciones_activasdif
# porcentaje_utilizacion
# quanto_mod
# valor_cuotas_codeudores_smlv
# valor_utilizado_smlv

In [1]:
"""
MODULE: processing_buro
This script extracts Buró information from the Buró file
Steps:
1. Get buro files
2. Get the needed columns and rows from Buro file including the needed historical period
3. Create additional columns
"""

import numpy as np
import pandas as pd
import s3fs
import pyarrow.parquet as pq

def process_buro(input_path1, input_path2):
    """Using the input_paths for the last two buro files, this function puts everything together"""

    # 1. Gets and appends the last two buro files
    buro = get_buro_file(input_path1, input_path2)
    print('get_buro_file')
    # 2. Get the needed columns and rows from buro file including the needed historical period
    buro = get_buro_info(buro)
    print('get_buro_info')
    # 3. Perform simple imputation to output data frame
    buro_final = create_buro_df(buro)
    print('create_buro_df')
    print('buro_df created successfully')

    return buro_final


def get_buro_file(input_path1, input_path2):
    """ Gets the input_paths of the last two the buro files, appends them and drops some not useful columns
     and outputs a DataFrame
    :param
    input_path1: location of the oldest buro file
    input path2: location of the most recent buro file
    :return: DataFrame
    """

    buro_1 = spark_read_parquet(input_path1)
    buro_1.columns = buro_1.columns.str.lower()
    buro_1 = buro_1.rename(columns={'cont_id': 'id_cliente',
                                    'ult_año_moras_90_cartera_hist':'ult_ano_moras_90_cartera_hist',
                                    'ult_año_moras_120_cartera_hist':'ult_ano_moras_120_cartera_hist'})

    buro_1['marca_buro'] = 1
    if buro_1['id_cliente'].isnull().sum() > 0:
        buro_1 = buro_1.loc[buro_1['id_cliente'].notnull()]

    buro_1['id_cliente'] = buro_1['id_cliente'].astype('str')
    buro_1['id_cliente'] = buro_1['id_cliente'].astype(np.int64)

    buro_2 = spark_read_parquet(input_path2)
    buro_2.columns = buro_2.columns.str.lower()
    buro_2 = buro_2.rename(columns={'cont_id': 'id_cliente',
                                    'ult_año_moras_90_cartera_hist':'ult_ano_moras_90_cartera_hist',
                                    'ult_año_moras_120_cartera_hist':'ult_ano_moras_120_cartera_hist'})

    buro_2['marca_buro'] = 2
    if buro_2['id_cliente'].isnull().sum() > 0:
        buro_2 = buro_2.loc[buro_2['id_cliente'].notnull()]

    buro_2['id_cliente'] = buro_2['id_cliente'].astype('str')
    buro_2['id_cliente'] = buro_2['id_cliente'].astype(np.int64)

    buro = buro_1.append(buro_2)
    if buro['id_cliente'].isnull().sum() > 0:
        buro = buro.loc[buro['id_cliente'].notnull()]

    buro['id_cliente'] = buro['id_cliente'].astype(np.int64)
    buro = buro[['id_cliente', 'fecha_envio', 'marca_buro', 'numero_obligaciones_activas',
                 'porcentaje_utilizacion',
                 'quanto_mod',
                 'valor_cuotas_codeudores',
                 'valor_utilizado',
                 'valor_cupos']]
    
    buro = buro.fillna(0)

    return buro


def get_buro_info(buro):
    """Deletes duplicates and casts dates
    :param buro to buro file location
    :return: DataFrame
    """
    # casting dates
    buro = buro[buro['fecha_envio'].notnull()]
    buro['fecha_envio'] = buro['fecha_envio'].astype('str')
    buro['fecha_buro'] = (
                buro['fecha_envio'].str.replace('-', '').str[0:4] + buro['fecha_envio'].str.replace('-', '').str[
                                                                    4:6]).astype('int')

    # deleting duplicates
    buro = buro.sort_values(['id_cliente', 'fecha_buro'])
    buro = buro.drop_duplicates(subset=['id_cliente', 'fecha_buro'], keep='first')

    return buro


def create_buro_df(buro):
    """Creates client-level variables final variables
    :param buro containing buro relevant rows and columns
    :return: DataFrame
    """
    # creating lag variables
    buro['llave_lag'] = np.where(buro['id_cliente'] == buro['id_cliente'].shift(1), 1, 0)

    # variable cupo mercado
    buro['ano_po'] = buro['fecha_buro'].astype(str).str[0:4].astype(int)
    buro['smlv'] = np.where(buro['ano_po'] == 2018, 781242,
                            np.where(buro['ano_po'] == 2019, 828116,
                                     np.where(buro['ano_po'] == 2020, 877803,
                                              np.nan)))
    
    if (buro['porcentaje_utilizacion'] > 0).sum()/buro.shape[0] > 0.9:
    
        buro['porcentaje_utilizacion'] = [float(str(x).replace('"','').replace(',','.')) for x in buro['porcentaje_utilizacion']]
        buro['porcentaje_utilizacion'] = buro['porcentaje_utilizacion'].astype('float')
    else:
        buro['porcentaje_utilizacion'] = np.where(buro['valor_cupos']>0, round(buro['valor_utilizado']/buro['valor_cupos'],4),0) 


    buro['valor_cuotas_codeudores_smlv']= buro['valor_cuotas_codeudores']/buro['smlv']
    buro['valor_utilizado_smlv']= buro['valor_utilizado']/buro['smlv']
    
    var = 'numero_obligaciones_activas'
 
    # creating lag variables lag
    buro['llave_lag'] = np.where(buro['id_cliente'] == buro['id_cliente'].shift(1), 1, 0)

    buro[var+'lag1']=np.where(buro['llave_lag']==1,buro[var].shift(1),np.nan)
    buro[var+'dif']=np.where( ((np.isnan(buro[var])) | (buro[var]==0)) & ((np.isnan(buro[var+'lag1'])) | (buro[var+'lag1']==0)) ,0,
                              np.where( ((np.isnan(buro[var])) | (buro[var]==0)) & ((buro[var+'lag1']>0)) ,-1,
                              np.where( ((np.isnan(buro[var+'lag1'])) | (buro[var+'lag1']==0)) & ((buro[var]>0)) ,1,
                              np.where( (buro[var+'lag1']>0) & (buro[var]>0) , (buro[var]-buro[var+'lag1'])/buro[var+'lag1'],99))))
    
    del buro[var+'lag1'], var
    
    #Last filter
    buro = buro[buro['marca_buro'] == 2]

    buro_out = buro[['id_cliente', 'fecha_buro', 
                     'numero_obligaciones_activasdif',
                    'porcentaje_utilizacion',
                    'quanto_mod',
                    'valor_cuotas_codeudores_smlv',
                    'valor_utilizado_smlv']]

    return buro_out

def spark_read_parquet(s3_url: str, **args):
    fs = s3fs.S3FileSystem()
    # Leyendo base
    dataset = pq.ParquetDataset(s3_url, filesystem=fs)
    table = dataset.read()
    dataframe = table.to_pandas()

    del dataset, table

    return dataframe


In [2]:
input_path1 = 's3://adl-refined-dev-popular/data_parquet/riesgos-creditos/buro-comportamiento-financiero/riesgos-creditos_buro-comportamiento-financiero_experian_M202003'
input_path2 = 's3://adl-refined-dev-popular/data_parquet/riesgos-creditos/buro-comportamiento-financiero/riesgos-creditos_buro-comportamiento-financiero_experian_M201911'


buro_mdt = process_buro(input_path2, input_path1)
#buro_mdt = get_buro_file(input_path2, input_path1)
#buro_mdt = get_buro_info(buro_mdt)

/home/jupyter/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


get_buro_file
get_buro_info
create_buro_df
buro_df created successfully


In [3]:
print(buro_mdt.shape)
buro_mdt.head()

(547061, 7)


,id_cliente,fecha_buro,numero_obligaciones_activasdif,porcentaje_utilizacion,quanto_mod,valor_cuotas_codeudores_smlv,valor_utilizado_smlv
377976,101052294884460201,202004,0.000000,0.1372,2765000.0,0.000000,0.699474
98220,101052295928775502,202004,0.000000,0.0885,12596000.0,1.384137,4.667334
214214,101052312312302702,202004,0.250000,1.0179,3800000.0,0.000000,7.758005
301968,101052312315900401,202004,0.400000,0.5875,2097000.0,0.000000,2.088168
455604,101052312911760301,202004,-0.090909,0.6279,6671000.0,0.000000,53.468717


In [6]:
buro_mdt.count()

id_cliente                        547061
fecha_buro                        547061
numero_obligaciones_activasdif    547061
porcentaje_utilizacion            547061
quanto_mod                        547061
valor_cuotas_codeudores_smlv      547061
valor_utilizado_smlv              547061
dtype: int64

In [7]:
buro_mdt.describe()

,id_cliente,fecha_buro,numero_obligaciones_activasdif,porcentaje_utilizacion,quanto_mod,valor_cuotas_codeudores_smlv,valor_utilizado_smlv
count,5.470610e+05,547061.0,547061.000000,547061.000000,5.470610e+05,547061.000000,547061.000000
mean,5.513203e+17,202004.0,0.218617,1.531394,4.230464e+06,0.919466,6.003931
std,2.597742e+17,0.0,0.430196,133.728395,3.177826e+06,82.289280,13.560153
min,1.010523e+17,202004.0,-0.900000,0.000000,0.000000e+00,0.000000,0.000000
25%,3.272523e+17,202004.0,0.000000,0.000000,2.387000e+06,0.000000,0.000000
50%,5.530566e+17,202004.0,0.000000,0.170400,3.256000e+06,0.000000,1.442237
75%,7.768533e+17,202004.0,0.333333,0.624700,4.977000e+06,0.000000,6.319186
max,9.999590e+17,202004.0,24.000000,56752.000000,1.957670e+08,34176.233164,606.338780


## Guardando Base

In [8]:
buro_mdt.to_parquet('s3://adl-refined-dev-popular/parquet/TC_adquisicion/buro_piloto',engine='pyarrow', index=False)

In [ ]:
a## No correr

In [14]:
input_path1 = 's3://adl-refined-dev-popular/data_parquet/riesgos-creditos/buro-comportamiento-financiero/riesgos-creditos_buro-comportamiento-financiero_experian_M201911'
input_path2 = 's3://adl-refined-dev-popular/data_parquet/riesgos-creditos/buro-comportamiento-financiero/riesgos-creditos_buro-comportamiento-financiero_experian_M201908'


In [15]:
buro_1 = spark_read_parquet(input_path1)
buro_1.columns = buro_1.columns.str.lower()
buro_1 = buro_1.rename(columns={'cont_id': 'id_cliente',
                                'ult_año_moras_90_cartera_hist':'ult_ano_moras_90_cartera_hist',
                                'ult_año_moras_120_cartera_hist':'ult_ano_moras_120_cartera_hist'})

buro_1['marca_buro'] = 1

buro_2 = spark_read_parquet(input_path2)
buro_2.columns = buro_2.columns.str.lower()
buro_2 = buro_2.rename(columns={'cont_id': 'id_cliente',
                                'ult_año_moras_90_cartera_hist':'ult_ano_moras_90_cartera_hist',
                                'ult_año_moras_120_cartera_hist':'ult_ano_moras_120_cartera_hist'})

buro_2['marca_buro'] = 2

buro = buro_1.append(buro_2)
if buro['id_cliente'].isnull().sum() > 0:
    buro = buro.loc[buro['id_cliente'].notnull()]

buro['id_cliente'] = buro['id_cliente'].astype(np.int64)

In [16]:
len(buro_2[buro_2['id_cliente'].isin(buro_1['id_cliente'])])/buro_2.shape[0]

0.8858533821859793

In [17]:
buro_1.dtypes

tipo_id                  object
rango_aproximado_edad    object
genero                   object
ciudad_de_expedicion     object
acierta_a_financiero      int32
                          ...  
fecha_envio              object
fecha_data               object
tipo_cliente             object
id_cliente               object
marca_buro                int64
Length: 132, dtype: object

In [18]:
buro_2.dtypes

tipo_id                  object
rango_aproximado_edad    object
genero                   object
ciudad_de_expedicion     object
acierta_a_financiero      int32
                          ...  
fecha_envio              object
fecha_data               object
tipo_cliente             object
id_cliente               object
marca_buro                int64
Length: 132, dtype: object

In [19]:
buro_1.columns

Index(['tipo_id', 'rango_aproximado_edad', 'genero', 'ciudad_de_expedicion',
       'acierta_a_financiero', 'quanto', 'quanto_mod',
       'porcen_de_cuotas_vs_ingreso', 'respuesta',
       'numero_obligaciones_activas',
       ...
       'cupo_segun_mercado', 'cupo_ajustadi', 'cupo_exp_fin', 'cupo_final',
       'experiencia_financiera', 'fecha_envio', 'fecha_data', 'tipo_cliente',
       'id_cliente', 'marca_buro'],
      dtype='object', length=132)

In [20]:
buro_2.columns

Index(['tipo_id', 'rango_aproximado_edad', 'genero', 'ciudad_de_expedicion',
       'acierta_a_financiero', 'quanto', 'quanto_mod',
       'porcen_de_cuotas_vs_ingreso', 'respuesta',
       'numero_obligaciones_activas',
       ...
       'cupo_segun_mercado', 'cupo_ajustadi', 'cupo_exp_fin', 'cupo_final',
       'experiencia_financiera', 'fecha_envio', 'fecha_data', 'tipo_cliente',
       'id_cliente', 'marca_buro'],
      dtype='object', length=132)

In [22]:
buro_1[['tdc_altura_maxima_de_mora',
'cartera_banca_alt_max_de_mora',
'cartera_coope_alt_max_de_mora',
'cartera_hipote_alt_max_de_mora']].describe()

,tdc_altura_maxima_de_mora,cartera_banca_alt_max_de_mora,cartera_coope_alt_max_de_mora,cartera_hipote_alt_max_de_mora
count,531576.000000,531576.000000,531576.000000,531576.000000
mean,0.274668,0.171432,0.021472,0.013486
std,1.169975,0.882151,0.304652,0.221806
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000
max,6.000000,6.000000,6.000000,6.000000


In [11]:
buro_1[['tdc_altura_maxima_de_mora',
'cartera_banca_alt_max_de_mora',
'cartera_coope_alt_max_de_mora',
'cartera_hipote_alt_max_de_mora']]

,tdc_altura_maxima_de_mora,cartera_banca_alt_max_de_mora,cartera_coope_alt_max_de_mora,cartera_hipote_alt_max_de_mora
0,None,None,None,None
1,None,None,None,None
2,None,None,None,None
3,None,None,None,None
4,None,None,None,None
...,...,...,...,...
547133,None,None,None,None
547134,None,None,None,None
547135,None,None,None,None
547136,None,None,None,None


In [30]:
buro_1.count()[buro_1.count() == 0]

ds_nombre_cliente                 0
quanto                            0
porcen_de_cuotas_vs_ingreso       0
respuesta                         0
porcentaje_utilizacion            0
rango_0                           0
rango_1                           0
rango_2                           0
rango_3                           0
rango_4                           0
rango_5                           0
rango_6                           0
tdc_altura_maxima_de_mora         0
cartera_banca_alt_max_de_mora     0
cartera_coope_alt_max_de_mora     0
cartera_hipote_alt_max_de_mora    0
endeudamiento                     0
utilizacion_sin_popular           0
util_sector_banca_sin_popular     0
util_sector_hip_sin_popular       0
cuotas_calculadas_tdc             0
cuotas_calculadas_créditos        0
cuotas_calculadas_hipotecarias    0
plazo_credito                     0
plazo_hipotecario                 0
tasa_hipotecario                  0
tasa_creditos                     0
plazo_tdc                   

In [29]:
buro_2.count()[buro_2.count() == 0]

Series([], dtype: int64)

In [9]:
buro_1[['fecha_envio','fecha_data']]

,fecha_envio,fecha_data
0,2020-04-28,2020-03-31
1,2020-04-28,2020-03-31
2,2020-04-28,2020-03-31
3,2020-04-28,2020-03-31
4,2020-04-28,2020-03-31
...,...,...
547133,2020-04-28,2020-03-31
547134,2020-04-28,2020-03-31
547135,2020-04-28,2020-03-31
547136,2020-04-28,2020-03-31


In [10]:
buro_2[['fecha_envio','fecha_data']]

,fecha_envio,fecha_data
0,2019-12-26,2019-11-30
1,2019-12-26,2019-11-30
2,2019-12-26,2019-11-30
3,2019-12-26,2019-11-30
4,2019-12-26,2019-11-30
...,...,...
531571,2019-12-26,2019-11-30
531572,2019-12-26,2019-11-30
531573,2019-12-26,2019-11-30
531574,2019-12-26,2019-11-30


In [11]:
buro_1.columns[~buro_1.columns.isin(buro_2.columns)]
#buro_2.columns[~buro_2.columns.isin(buro_1.columns)]

Index(['no_identificacion', 'ds_nombre_cliente'], dtype='object')

In [13]:
buro_1[['id_cliente', 'fecha_envio', 'numero_obligaciones_activas',
                 'porcentaje_utilizacion',
                 'quanto_mod',
                 'valor_cuotas_codeudores',
                 'valor_utilizado']].count()

id_cliente                     547138
fecha_envio                    547138
numero_obligaciones_activas    547138
porcentaje_utilizacion              0
quanto_mod                     547138
valor_cuotas_codeudores        547138
valor_utilizado                547138
dtype: int64

In [14]:
buro_2[['id_cliente', 'fecha_envio', 'numero_obligaciones_activas',
                 'porcentaje_utilizacion',
                 'quanto_mod',
                 'valor_cuotas_codeudores',
                 'valor_utilizado']].count()


id_cliente                     531570
fecha_envio                    531576
numero_obligaciones_activas    531576
porcentaje_utilizacion         531576
quanto_mod                     531576
valor_cuotas_codeudores        531576
valor_utilizado                531576
dtype: int64

In [20]:
dat = pd.read_csv('s3://adl-refined-dev-popular/data_orig/riesgos-creditos/buro-comportamiento-financiero/riesgos-creditos_buro-comportamiento-financiero_experian_M202003.csv',sep='|', encoding='latin-1')
dat.head()

,NO_IDENTIFICACION,DS_NOMBRE_CLIENTE,RANGO_APROXIMADO_EDAD,GENERO,CIUDAD_DE_EXPEDICION,ACIERTA_A_FINANCIERO,QUANTO,QUANTO_MOD,PORCEN_DE_CUOTAS_VS_INGRESO,RESPUESTA,...,CUPO_SUGERIDO_2,CUPO_SUGERIDO_3,CUPO_SEGUN_MERCADO,CUPO_AJUSTADI,CUPO_EXP_FIN,CUPO_FINAL,EXPERIENCIA_FINANCIERA,FECHA_ENVIO,FECHA_DATA,ID_CLIENTE
0,10000001,NaN,72.0,M,PEREIRA,752,NaN,3528000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28/04/2020,31/03/2020,193655260482175901
1,10000022,NaN,43.0,M,PEREIRA,828,NaN,12754000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28/04/2020,31/03/2020,806756468325404502
2,10000075,NaN,43.0,M,PEREIRA,936,NaN,4073000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28/04/2020,31/03/2020,644055599166846201
3,10000134,NaN,43.0,M,PEREIRA,702,NaN,5363000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28/04/2020,31/03/2020,932453306201419701
4,10000139,NaN,43.0,M,PEREIRA,780,NaN,5898000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28/04/2020,31/03/2020,531152295443333302


In [ ]:
valor_utilizado/valor_de_cupos

In [51]:
buro_2.columns[buro_2.columns.str.contains('cupo')]

Index(['valor_cupos', 'cupo_sin_popular', 'max_cupo_tdc_sin_popular',
       'promedio_cupo_tdc_sin_popular', 'cuposectorbancario_sin_popular',
       'maxcuposectorbancasin_popular', 'promcuposectorbancasin_popular',
       'cupo_sector_hip_sin_popular', 'max_cupo_sector_hip_sin_popu',
       'prom_cupo_sector_hip_sin_popu', 'cupo_sugerido_1', 'cupo_sugerido_2',
       'cupo_sugerido_3', 'cupo_segun_mercado', 'cupo_ajustadi',
       'cupo_exp_fin', 'cupo_final'],
      dtype='object')

In [22]:
dat[['ID_CLIENTE', 'FECHA_ENVIO',  'PORCENTAJE_UTILIZACION',
                 ]].count()

ID_CLIENTE                547138
FECHA_ENVIO               547138
PORCENTAJE_UTILIZACION         0
dtype: int64

In [40]:
buro_2['rango_0'].value_counts()

0    531378
1       196
2         2
Name: rango_0, dtype: int64

In [61]:
(buro_2['porcentaje_utilizacion'].notnull()).sum()/buro_2.shape[0]

1.0

In [63]:

buro_2['porcentaje_utilizacion2'] = np.where(buro_2['valor_cupos'].isnull(),np.nan,
                                             np.where(buro_2['valor_cupos']>0, round(buro_2['valor_utilizado']/buro_2['valor_cupos'],2)
                                                      ,0)
                                            ) 
print((buro_2['porcentaje_utilizacion2'] - buro_2['porcentaje_utilizacion']).describe())


buro_2[['valor_utilizado', 'valor_cuotas', 'porcentaje_utilizacion2','porcentaje_utilizacion']]

count    5.315760e+05
mean    -7.330090e-07
std      5.052510e-04
min     -1.000010e-02
25%     -1.192093e-09
50%      0.000000e+00
75%      2.384186e-09
max      1.000006e-02
dtype: float64


,valor_utilizado,valor_cuotas,porcentaje_utilizacion2,porcentaje_utilizacion
0,914000,162000,0.04,0.04
1,0,0,0.00,0.00
2,0,0,0.00,0.00
3,12026000,0,0.81,0.81
4,466000,248000,0.23,0.23
...,...,...,...,...
531571,447000,330000,0.18,0.18
531572,0,0,0.00,0.00
531573,1079000,201000,0.03,0.03
531574,0,0,0.00,0.00


In [65]:
dat[['VALOR_UTILIZADO', 'VALOR_CUPOS']]

,VALOR_UTILIZADO,VALOR_CUPOS
0,8178000.0,7954000.0
1,10677000.0,30338000.0
2,4794000.0,29910000.0
3,NaN,NaN
4,1723000.0,3700000.0
...,...,...
547133,0.0,5968000.0
547134,460000.0,23000000.0
547135,NaN,NaN
547136,NaN,NaN


In [66]:
dat['PORCENTAJE_UTILIZACION'] = np.where(dat['VALOR_CUPOS'].isnull(),np.nan,
                                             np.where(dat['VALOR_CUPOS']>0, round(dat['VALOR_UTILIZADO']/dat['VALOR_CUPOS'],2)
                                                      ,0)
                                            ) 

dat[['VALOR_UTILIZADO', 'VALOR_CUPOS', 'PORCENTAJE_UTILIZACION']]

,VALOR_UTILIZADO,VALOR_CUPOS,PORCENTAJE_UTILIZACION
0,8178000.0,7954000.0,1.03
1,10677000.0,30338000.0,0.35
2,4794000.0,29910000.0,0.16
3,NaN,NaN,NaN
4,1723000.0,3700000.0,0.47
...,...,...,...
547133,0.0,5968000.0,0.00
547134,460000.0,23000000.0,0.02
547135,NaN,NaN,NaN
547136,NaN,NaN,NaN


In [31]:
dat.count()[dat.count() == 0]

DS_NOMBRE_CLIENTE                 0
QUANTO                            0
PORCEN_DE_CUOTAS_VS_INGRESO       0
RESPUESTA                         0
PORCENTAJE_UTILIZACION            0
RANGO_0                           0
RANGO_1                           0
RANGO_2                           0
RANGO_3                           0
RANGO_4                           0
RANGO_5                           0
RANGO_6                           0
TDC_ALTURA_MAXIMA_DE_MORA         0
CARTERA_BANCA_ALT_MAX_DE_MORA     0
CARTERA_COOPE_ALT_MAX_DE_MORA     0
CARTERA_HIPOTE_ALT_MAX_DE_MORA    0
ENDEUDAMIENTO                     0
UTILIZACION_SIN_POPULAR           0
UTIL_SECTOR_BANCA_SIN_POPULAR     0
UTIL_SECTOR_HIP_SIN_POPULAR       0
CUOTAS_CALCULADAS_TDC             0
CUOTAS_CALCULADAS_CRÃDITOS       0
CUOTAS_CALCULADAS_HIPOTECARIAS    0
PLAZO_CREDITO                     0
PLAZO_HIPOTECARIO                 0
TASA_HIPOTECARIO                  0
TASA_CREDITOS                     0
PLAZO_TDC                   